In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() #? load the API key in the .env file

#Import the required libraries
import requests
import json

In [ ]:
# Base URL for Nutritionix natural language API
url = "https://trackapi.nutritionix.com/v2/natural/nutrients"

APP_ID = os.getenv("NUTRITIONIX_APP_ID")
API_KEY = os.getenv("NUTRITIONIX_API_KEY")

# Define headers with your API credentials
headers = {
    "x-app-id": APP_ID,
    "x-app-key": API_KEY,
    "Content-Type": "application/json"
}

# Define the query parameters
query = "1 cup of rice and 2 eggs"

# Send POST request with food query
response = requests.post(url, headers=headers, json={"query": query})

# Check if request was successful
if response.status_code == 200:
    data = response.json()
    print("Nutrition data for query:", query)
    for food in data['foods']:
        print(f"Food: {food['food_name']}")
        print(f"Calories: {food['nf_calories']} kcal")
        print(f"Protein: {food['nf_protein']} g")
        print(f"Carbohydrates: {food['nf_total_carbohydrate']} g")
        print(f"Fats: {food['nf_total_fat']} g\n")
else:
    print("Error:", response.status_code, response.text)

Nutrition data for query: 1 cup of rice and 2 eggs
Food: rice
Calories: 205.4 kcal
Protein: 4.25 g
Carbohydrates: 44.51 g
Fats: 0.44 g

Food: eggs
Calories: 143 kcal
Protein: 12.56 g
Carbohydrates: 0.72 g
Fats: 9.51 g



In [45]:
# create a object that sign in to the nutritionix API and get the data
class NutritionixAPI:
    def __init__(self):
        self.url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
        self.headers = {
            "x-app-id": APP_ID,
            "x-app-key": API_KEY,
            "Content-Type": "application/json"
        }

    def get_nutrition_data(self, query):
        response = requests.post(self.url, headers=self.headers, json={"query": query})
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return None
        
    def get_nutrition_info(self, query):
        data = self.get_nutrition_data(query)
        if data:
            info = []
            for food in data['foods']:
                info.append({
                    "food": food['food_name'],
                    "calories": food['nf_calories'],
                    "protein": food['nf_protein'],
                    "carbs": food['nf_total_carbohydrate'],
                    "fats": food['nf_total_fat']
                })
            return info
        else:
            return None
    def get_summary(self, query):
        data = self.get_nutrition_data(query)
        summeries = {}
        for food in data['foods']:
            # Summerize the nutrition data
            for key, value in food.items():
                if key in ["nf_calories", "nf_protein", "nf_total_carbohydrate", "nf_total_fat"]:
                    if key in summeries:
                        summeries[key] += value
                    else:
                        summeries[key] = value
        # rename the keys
        summeries["calories"] = summeries.pop("nf_calories")
        summeries["protein"] = summeries.pop("nf_protein")
        summeries["carbs"] = summeries.pop("nf_total_carbohydrate")
        summeries["fats"] = summeries.pop("nf_total_fat")

        # round the values to 2 decimal places
        for key, value in summeries.items():
            summeries[key] = round(value, 2)
        return summeries

In [51]:
# create a object of the NutritionixAPI class
api = NutritionixAPI()

# get the nutrition data for the query
data = api.get_nutrition_info("1 cup of rice and 2 eggs")

# print the nutrition data
if data:
    for food in data:
        print(f"Food: {food['food']}")
        print(f"Calories: {food['calories']} kcal")
        print(f"Protein: {food['protein']} g")
        print(f"Carbohydrates: {food['carbs']} g")
        print(f"Fats: {food['fats']} g\n")
else:
    print("Error getting nutrition data")

data_all=api.get_summary("1 cup of rice and 2 eggs")
print("Summary:",data_all, "\n")

data_other_test=api.get_nutrition_data("1 Pizza Margehrita and 1 Glass of Coke")
if data_other_test:
    for food in data_other_test['foods']:
        print(f"Food: {food['food_name']}")
        print(f"Calories: {food['nf_calories']} kcal")
        print(f"Protein: {food['nf_protein']} g")
        print(f"Carbohydrates: {food['nf_total_carbohydrate']} g")
        print(f"Fats: {food['nf_total_fat']} g\n")

Food: rice
Calories: 205.4 kcal
Protein: 4.25 g
Carbohydrates: 44.51 g
Fats: 0.44 g

Food: eggs
Calories: 143 kcal
Protein: 12.56 g
Carbohydrates: 0.72 g
Fats: 9.51 g

Summary: {'calories': 348.4, 'protein': 16.81, 'carbs': 45.23, 'fats': 9.95} 

Food: pizza
Calories: 284.62 kcal
Protein: 12.19 g
Carbohydrates: 35.66 g
Fats: 10.37 g

Food: coke
Calories: 103.15 kcal
Protein: 0 g
Carbohydrates: 25.44 g
Fats: 0.61 g

